In [1]:
import pandas as pd
import urllib
import os
import string
import re
import numpy as np

In [2]:
df=pd.read_csv("Data/noms_voies_actuelles_paris.csv", sep=';')
df.head()

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,LONG,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,Orig. du nom. ~ Pierre Jean Baptiste Gerbier (...,13.0,174.0,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis."
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"Orig. du nom. ~ M. Joly, propriétaire.",12.0,135.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"Orig. du nom. ~ M. Léchevin, propriétaire.\r\n...",15.0,73.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis."
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"Orig. du nom. ~ François Richard (1765-1839), ...",60.0,1500.0,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla..."
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"Orig. du nom. ~ Baruch Spinoza (1632-1677), ph...",12.0,77.0,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81."


In [3]:
len(df)

6475

# 1_Cleaning streets names

On va utiliser la variable "HISTORIQUE", pour extraire des informations relatives à la personne qui a donné son nom à la voie.

In [4]:
# Cleaner à la main : ceux erreurs
#df['HISTORIQUE'][25] = "Grace Murray Hopper (New-York, 1906- comté d'Arlington, 1992), mathématicienne et informaticienne américaine, ayant mis au point le premier compilateur ainsi que le langage Cobol. \r\nHistorique. ~ voie créée dans le secteur de la Halle Freyssinet, précédemment voie GR/13. Anciennement partie de la voie GM/13."

In [5]:
# Removing expressions "Origin. du nom"
removed_expressions = ["Orig. du nom. ~ ", "Orig.du nom.~ ", "Orig. Du nom. ~"]
for expression in removed_expressions :
    df["HISTORIQUE"] = df['HISTORIQUE'].astype(str).apply(lambda x: x.replace(expression, ''))

In [6]:
new = df['HISTORIQUE'].str.split("\r\nHistorique. ~ |Historique. ~ ", n = 1, expand = True) 
df['Histoire'] = new[1]
df['Rues'] = new[0]

In [7]:
new = df["Rues"].str.split(";", n = 1, expand = True)
df['Quartier'] = new[1]
df['Rues'] = new[0]

In [8]:
df

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,LONG,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN,Histoire,Rues,Quartier
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,"Pierre Jean Baptiste Gerbier (1725-1788), avoc...",13.00,174.0,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis.",None,"Pierre Jean Baptiste Gerbier (1725-1788), avocat",quartier où ont été groupés des noms de juris...
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"M. Joly, propriétaire.",12.00,135.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse,None,"M. Joly, propriétaire.",None
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"M. Léchevin, propriétaire.\r\nHistorique. ~ Av...",15.00,73.0,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis.",Avant 1942 : passage Léchevin. La partie de ce...,"M. Léchevin, propriétaire.",None
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"François Richard (1765-1839), manufacturier d'...",60.00,1500.0,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla...",Cette voie qui recouvre le canal Saint-Martin ...,"François Richard (1765-1839), manufacturier d'...",quartier de fabriques.
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"Baruch Spinoza (1632-1677), philosophe holland...",12.00,77.0,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81.",None,"Baruch Spinoza (1632-1677), philosophe holland...",None
5,NaN,4865,Quinze-Vingts.,rue,NaN,jean bouton,rue Jean Bouton,4775,"M. Jean Bouton, propriétaire.\r\nHistorique. ~...",13.00,98.0,"Classement, alignements (retenus au PAZ de la ...",XIIe,voie publique,"boulevard Diderot, 30, place Henri Frenay et r...","Rue Paul-Henri Grauwin, 22.",Dénommée impasse Jean Bouton avant 1929. Elle ...,"M. Jean Bouton, propriétaire.",None
6,NaN,2320,Quinze-Vingts.,avenue,de,corbera,avenue de Corbera,2321,Nom de propriétaire.\r\nHistorique. ~ Précédem...,15.00,102.0,"Classement, alignements (non retenus au POS et...",XIIe,voie publique,"rue de Charenton, 131-135.","rue Crozatier, 11-17.","Précédemment, passage Guillaumot Lainet ; plus...",Nom de propriétaire.,None
7,1965-05-05T01:00:00+01:00,6844,Bel Air.,rue,de la,nouvelle-calédonie,rue de la Nouvelle-Calédonie,6787,Ile de la Mélanésie ; territoire d'outre-mer.\...,20.00,120.0,Dénomination. ~ Arr. du 5 mai 1965.,XIIe,voie publique,"boulevard Soult, 20.","rue du Général Archinard, avenue du Général Me...",Ouverte par la Ville de Paris sur l'emplacemen...,Ile de la Mélanésie,territoire d'outre-mer.
8,NaN,R530,Picpus.,avenue,des,canadiens,avenue des Canadiens,Z530,Un hôpital militaire accueillant principalemen...,10.00,660.0,Observation. ~ Le premier tronçon est situé da...,XIIe,voie publique,1° tronçon: situé le long de la limite de Pari...,2° tronçon: situé entre la rue du Maréchal Lec...,Le premier tronçon était situé sur les territo...,Un hôpital militaire accueillant principalemen...,None
9,1911-03-21T01:00:00+01:00,2904,Picpus.,avenue,NaN,dorian,avenue Dorian,2927,"Pierre Frédéric Dorian (1814-1873), manufactur...",40.00,80.0,Classement. ~ Déc. du 23 septembre 1881.\r\nAl...,XIIe,voie publique,"rue de Picpus, 9.","place de la Nation, 4.",Précédemment rue Dorian (déc. du 25 janvier 18...,"Pierre Frédéric Dorian (1814-1873), manufactur...",None


## 1_1 Distinction noms de personnes

On distingue les rues qui portent des noms d'individus et les autres.

Plusieurs façons de reconnaître les rues qui portent des noms d'individus :
- On a souvent les dates de naissance et mort entre parenthèses pour les rues qui portent des noms de personnes
- S'il s'agit d'un propriétaire (qui possédait donc des immeubles dans cette rue), on peut filter avec "propriétaire"
- Pour certaines rues (plutôt des impasses), on n'a pas la description de la personne, car il s'agit d'une rue au voisinage d'un boulevard par exemple (et l'information sur la personne est disponible à la ligne correspondant au boulevard)
- Ou sinon, si on n'a ni l'un ni l'autre, on peut utiliser la liste des prénoms (puisque pour certains individus, on n'a ni les autres informations ci-dessus)

### 1. 1. 1. Voisinages

In [9]:
# dataframe voisinages: on remplace la "Rues" par la vraie référence à la rue
voisinages = df.loc[df['Rues'].str.startswith("Voisinage")].reset_index()
voisinages["Voisinage rue"] = voisinages["Rues"]
voisinages = pd.merge(voisinages.loc[:, voisinages.columns != 'Rues'], 
                      df.loc[~df['Rues'].str.contains("Voisinage"), ['Rues', 'NOM de VOIE']], 
                      how='left', 
                      on='NOM de VOIE')

In [10]:
# Issue: pour certaines rues, on n'a pas exactement le même Libellé voie entre la rue principale et celle au voisinage. 
# Ou sinon, il est juste inscrit "voisinage", même pour la rue principale (si voisinage d'une église par ex)
# On va corriger ça.
list_voisinages_pb = voisinages.loc[voisinages['Rues'].isna(), "NOM de VOIE"].tolist()

for voisinage in list_voisinages_pb :
    if len(df.loc[df['NOM de VOIE'].str.contains(voisinage)]) == len(df.loc[df['NOM de VOIE'].str.startswith(voisinage)]) :
        voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]
    
    elif len(df.loc[df['NOM de VOIE'].str.contains(voisinage)])==1:
        voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]

    elif len(df.loc[df['NOM de VOIE'].str.contains(voisinage)])==2:
        new_historique_rue = df.loc[(~df['Rues'].str.contains("Voisinage")) & (df['NOM de VOIE'].str.contains(voisinage)), 'Rues']
        if new_historique_rue.empty :
            voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]
        else :
            voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = new_historique_rue.values[0]
    
    else :
        voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Rues"] = voisinages.loc[voisinages['NOM de VOIE']==voisinage, "Voisinage rue"]

In [11]:
voisinages = voisinages.drop_duplicates(subset='index', keep="first")
voisinages = voisinages.set_index("index")
voisinages.index.name=None

In [12]:
df = df.merge(voisinages.loc[:, ['Rues']], how='left', left_index=True, right_index=True)

In [13]:
df.loc[~df['Rues_y'].isna(), 'Rues_x'] = df['Rues_y']
df = df.drop(['Rues_y'], axis=1).rename(columns={'Rues_x' : 'Rues'})

### 1. 1. 2. Labeliser: si c'est une personne

In [14]:
# Dates de naissance et de mort + propriétaire
df['Personne'] = 0
df.loc[df['HISTORIQUE'].str.contains("\(|, propriétaire|Nom de propriétaire"), ['Personne']] = 1

In [15]:
# Ajouter les saints (pas toujours ajouté)
df.loc[df['NOM de VOIE'].str.contains("saint"), ['Personne']] = 1

In [16]:
df['Personne'].value_counts()

1    3961
0    2514
Name: Personne, dtype: int64

In [17]:
# On ne les a pas tous. Exemple:
df[df.index==6446]

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,LONG,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN,Histoire,Rues,Quartier,Personne
6446,1864-08-24T00:53:28+00:53,4229,Place Vendôme.,rue,NaN,gomboust,rue Gomboust,4183,"Jacques Gomboust, ingénieur du Roi, auteur d'u...",10.0,60.0,Alignements (non retenus au POS et non repris ...,Ier,voie publique,"rue Saint-Roch, 57.","place du Marché Saint-Honoré, 38.","Précédemment, rue de la Corderie Saint-Honoré ...","Jacques Gomboust, ingénieur du Roi, auteur d'u...",voisinage de l'Hôtel du Saint-Esprit où ce pl...,0


# 2. Ajouter genre

In [18]:
for col in ['NOM de VOIE', 'HISTORIQUE', 'Rues']:
    df[col] = df[col].str.lower()

In [19]:
names = pd.read_csv("Data/Dico_gender_proportion.csv").drop('index', axis=1)
prenoms_list = names['prenoms'].tolist()

In [20]:
df['Sexe'] = float('nan')
for prenom in prenoms_list :
    sexe = names.loc[names['prenoms']==prenom, 'sexe'].values[0]
    df.loc[(df['Personne']==1) & (df['Rues'].str.contains(prenom)), 'Sexe'] = sexe
    df.loc[(df['Personne']==1) & (df['HISTORIQUE'].str.contains(prenom) & df['Sexe'].isna()), 'Sexe'] = sexe

# Pour les propriétaires :
df.loc[(df['Personne']==1) & (df['HISTORIQUE'].str.startswith('m.') & df['HISTORIQUE'].str.contains('propriétaire')), 'Sexe'] = 'M'
df.loc[(df['Personne']==1) & (df['HISTORIQUE'].str.contains('nom de propriétaire')), 'Sexe'] = 'M'

# Les saints :
df.loc[(df['Personne']==1) & (df['HISTORIQUE'].str.contains('sainte')), 'Sexe'] = 'F'
df.loc[(df['Personne']==1) & (df['HISTORIQUE'].str.contains('saint') & df['Sexe'].isna()), 'Sexe'] = 'M'

# Autres
df.loc[df['Personne']==0 , 'Sexe'] = 'Autre'

In [21]:
df['Sexe'].value_counts()

M        2890
Autre    2514
F        1001
Name: Sexe, dtype: int64

In [22]:
df

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,...,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN,Histoire,Rues,Quartier,Personne,Sexe
0,1864-02-03T00:53:28+00:53,4166,Roquette.,rue,NaN,gerbier,rue Gerbier,4112,"pierre jean baptiste gerbier (1725-1788), avoc...",13.00,...,Ouverture et alignements (non retenus au POS e...,XIe,voie publique,"rue de la Folie-Regnault, 15.","rue de la Roquette, 168 bis.",None,"pierre jean baptiste gerbier (1725-1788), avocat",quartier où ont été groupés des noms de juris...,1,M
1,NaN,4994,Saint-Ambroise.,cité,NaN,joly,cité Joly,4886,"m. joly, propriétaire.",12.00,...,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"rue du Chemin Vert, 121.",en impasse,None,"m. joly, propriétaire.",None,1,M
2,NaN,5441,Saint-Ambroise.,rue,NaN,léchevin,rue Léchevin,5376,"m. léchevin, propriétaire.\r\nhistorique. ~ av...",15.00,...,"Classement, alignements (non retenus au POS et...",XIe,voie publique,"avenue Parmentier, 64.","passage Saint-Ambroise, 9 bis.",Avant 1942 : passage Léchevin. La partie de ce...,"m. léchevin, propriétaire.",None,1,M
3,NaN,8208,Folie-Méricourt. Saint-Ambroise. Roquette.,boulevard,NaN,richard lenoir,boulevard Richard Lenoir,8197,"françois richard (1765-1839), manufacturier d'...",60.00,...,Alignements (non retenus au POS et non repris ...,XIe,voie publique,"boulevard Beaumarchais, 2 et place de la Basti...","avenue de la République, 22, rue Rampon et pla...",Cette voie qui recouvre le canal Saint-Martin ...,"françois richard (1765-1839), manufacturier d'...",quartier de fabriques.,1,M
4,1885-10-11T00:53:28+00:53,9057,Saint-Ambroise.,rue,NaN,spinoza,rue Spinoza,8671,"baruch spinoza (1632-1677), philosophe holland...",12.00,...,"Ouverture, alignements (non retenus au POS et ...",XIe,voie publique,"avenue de la République, 103.","boulevard de Ménilmontant, 81.",None,"baruch spinoza (1632-1677), philosophe holland...",None,1,NaN
5,NaN,4865,Quinze-Vingts.,rue,NaN,jean bouton,rue Jean Bouton,4775,"m. jean bouton, propriétaire.\r\nhistorique. ~...",13.00,...,"Classement, alignements (retenus au PAZ de la ...",XIIe,voie publique,"boulevard Diderot, 30, place Henri Frenay et r...","Rue Paul-Henri Grauwin, 22.",Dénommée impasse Jean Bouton avant 1929. Elle ...,"m. jean bouton, propriétaire.",None,1,M
6,NaN,2320,Quinze-Vingts.,avenue,de,corbera,avenue de Corbera,2321,nom de propriétaire.\r\nhistorique. ~ précédem...,15.00,...,"Classement, alignements (non retenus au POS et...",XIIe,voie publique,"rue de Charenton, 131-135.","rue Crozatier, 11-17.","Précédemment, passage Guillaumot Lainet ; plus...",nom de propriétaire.,None,1,M
7,1965-05-05T01:00:00+01:00,6844,Bel Air.,rue,de la,nouvelle-calédonie,rue de la Nouvelle-Calédonie,6787,ile de la mélanésie ; territoire d'outre-mer.\...,20.00,...,Dénomination. ~ Arr. du 5 mai 1965.,XIIe,voie publique,"boulevard Soult, 20.","rue du Général Archinard, avenue du Général Me...",Ouverte par la Ville de Paris sur l'emplacemen...,ile de la mélanésie,territoire d'outre-mer.,0,Autre
8,NaN,R530,Picpus.,avenue,des,canadiens,avenue des Canadiens,Z530,un hôpital militaire accueillant principalemen...,10.00,...,Observation. ~ Le premier tronçon est situé da...,XIIe,voie publique,1° tronçon: situé le long de la limite de Pari...,2° tronçon: situé entre la rue du Maréchal Lec...,Le premier tronçon était situé sur les territo...,un hôpital militaire accueillant principalemen...,None,0,Autre
9,1911-03-21T01:00:00+01:00,2904,Picpus.,avenue,NaN,dorian,avenue Dorian,2927,"pierre frédéric dorian (1814-1873), manufactur...",40.00,...,Classement. ~ Déc. du 23 septembre 1881.\r\nAl...,XIIe,voie publique,"rue de Picpus, 9.","place de la Nation, 4.",Précédemment rue Dorian (déc. du 25 janvier 18...,"pierre frédéric dorian (1814-1873), manufactur...",None,1,M


In [23]:
pd.crosstab(df['TYPE DE VOIE'], df['Sexe'])

Sexe,Autre,F,M
TYPE DE VOIE,,,
allée,44,26,52
arcades,1,0,0
autoroute,2,0,0
avenue,80,84,184
belvédère,0,0,1
boulevard,23,26,66
carrefour,14,1,0
chaussée,2,0,0
chemin,14,0,0


# Suite

A corriger :
- Certaines rues ne sont pas labelisées comme personnes (ex : 6446)
- Certaines rues le sont alors qu'elles ne sont pas des personnes (ex : 6458, il faut bien corriger pour parenthèses + chiffres) ; aussi bd des italiens.
- Dans le genre, pb si prénom étrange (comme Spinoza) : utiliser liste des célébrités
- Certaines erreurs, quand des femmes portent un nom de famille d'homme (ex : 24, car place Marcelle Henry). IL faut modifier l'attribution des noms: looper pour distinguer nom et prénom (avec la position dans le texte peut-être). L'inverse aussi (4986, Garibaldi labelisé comme une femme).

Pour les voisinages : comme on cherche maintenant dans "HISTORIQUE". Il faut remodifier pour chercher dans "Rues" quand c'est un voisinage (créer une autre variable).
Autre idée: récupérer via Wikipédia (liste des rues portant des noms de femmes.

In [24]:
# Toujours pas parfait du tout :
df.loc[(df['TYPE DE VOIE']=="boulevard") & (df['Sexe']=='F')] 

,DATE ARRET,CDGI,QUARTIER,TYPE DE VOIE,PREFIXE de VOIE,NOM de VOIE,Libellé VOIE,CVOIE,HISTORIQUE,LARGEUR,...,DETAIL,ARRONDISSEMENT,STATUT PRESUME,DEBUT,FIN,Histoire,Rues,Quartier,Personne,Sexe
230,1864-08-19T00:53:28+00:53,7006,Saint-Germain l'Auxerrois. Notre-Dame.,boulevard,du,palais,boulevard du Palais,6918,situé devant le palais de justice.\r\nhistoriq...,30.0,...,Alignements (retenus au POS et non repris au P...,"Ier,IVe",voie publique,quai de la Corse et quai de l'Horloge.,"quai du Marché Neuf, 8 et quai des Orfèvres.","Précédemment, partie du boulevard de Sébastopo...",situé devant le palais de justice.,None,1,F
471,1873-10-11T00:53:28+00:53,9623,Necker.,boulevard,de,vaugirard,boulevard de Vaugirard,9670,longe l'ancienne commune de vaugirard.\r\nhist...,42.0,...,Ouverture. ~ Ord. du Bureau des Finances du 16...,XVe,voie publique,"places Raoul Dautry, 2, et Bienvenüe, 6.","boulevard Pasteur, 71.",Précédemment : 1° à l'extérieur de l'ancien mu...,longe l'ancienne commune de vaugirard.,None,1,F
684,1931-04-21T01:00:00+01:00,9982,Ternes.,boulevard,de l',yser,boulevard de l'Yser,9978,"fleuve côtier de belgique, sur les rives duque...",26.0,...,Alignements (non retenus au POS et non repris ...,XVIIe,voie publique,"rue Claude Debussy, 3, et avenue de la Porte d...","avenue de la Porte de Villiers, 10-16.","Cette voie, précédemment comprise dans la rout...","fleuve côtier de belgique, sur les rives duque...",None,1,F
695,1864-08-19T00:53:28+00:53,8690,Jardin des Plantes. Salpêtrière.,boulevard,NaN,saint-marcel,boulevard Saint-Marcel,8912,"traverse l'ancien quartier saint-marcel, qui d...",40.0,...,Ouverture. ~ Déc. du 17 octobre 1857 (UP).\r\n...,"Ve,XIIIe",voie publique,"boulevard de l'Hôpital, 42.","avenue des Gobelins, 23.",None,"traverse l'ancien quartier saint-marcel, qui d...",None,1,F
770,1855-09-25T00:53:28+00:53,8894,Saint-Germain l'Auxerrois. Halles. Bonne Nouve...,boulevard,de,sébastopol,boulevard de Sébastopol,8525,port de guerre de crimée pris par l'armée angl...,30.0,...,Ouverture et alignements (non retenus au POS e...,"Ier,IIe,IIIe,IVe",voie publique,"avenue Victoria, 12.","boulevard Saint-Denis, 11.",Précédemment boulevard du Centre et partie de ...,port de guerre de crimée pris par l'armée angl...,None,1,F
1028,1931-04-21T01:00:00+01:00,2809,Ternes.,boulevard,de,dixmude,boulevard de Dixmude,2829,ville belge sur l'yser où les fusiliers marins...,26.0,...,Alignements (non retenus au POS et non repris ...,XVIIe,voie publique,"avenue de la Porte de Villiers, 11.",avenue de Salonique.,"Cette voie, précédemment comprise dans la rout...",ville belge sur l'yser où les fusiliers marins...,None,1,F
1185,1870-10-25T00:53:28+00:53,9907,Folie-Méricourt. Saint-Ambroise. Roquette. Sai...,boulevard,NaN,voltaire,boulevard Voltaire,9872,"françois marie arouet, dit voltaire (1694-1778...",30.0,...,Ouverture. ~ Déc. du 29 août 1857 (UP).\r\nAli...,XIe,voie publique,"place de la République, 4.","place de la Nation, 3.",Précédemment boulevard du Prince Eugène (déc. ...,"françois marie arouet, dit voltaire (1694-1778...",None,1,F
1221,NaN,4758,Gaillon. Vivienne. Chaussée d'Antin. Faubourg ...,boulevard,des,italiens,boulevard des Italiens,4687,voisinage de l'ancien théâtre de la comédie it...,35.0,...,Ouverture. ~ Lettres patentes de juillet 1676 ...,"IIe,IXe",voie publique,"rue de Richelieu, 103, et boulevard Haussmann, 1.","rue Louis le Grand, 34, et rue de la Chaussée ...","Appelé à l'origine boulevard Neuf, puis boulev...",voisinage de l'ancien théâtre de la comédie it...,None,1,F
1531,NaN,8619,Saint-Victor. Sorbonne. Monnaie. Odéon. Saint-...,boulevard,NaN,saint-germain,boulevard Saint-Germain,8845,traverse l'ancien faubourg saint-germain.\r\nh...,30.0,...,"Ouverture. ~ Déc. du 11 août 1855 (UP), entre ...","Ve,VIe,VIIe",voie publique,"rue des Fossés Saint-Bernard, 2, et quai de la...","rue Aristide Briand, 1, et quai Anatole France...",Le boulevard Saint-Germain a englobé ou suppri...,traverse l'ancien faubourg saint-germain.,N